# Métodos para buscar raíces

## Método de bisección 
Para aplicar este método consideremos tres sucesiones $a_n \le p_n \le b_n$ definidas por las siguientes relaciones

$$ p_n = \frac{a_n+b_n}{2},
\quad a_{n+1} = \begin{cases}
a_n & \mbox{si } f(a_n)\cdot f(p_n) <0 \\
p_n & \mbox{si } f(a_n)\cdot f(p_n) > 0\end{cases},
\quad b_{n+1} = \begin{cases}
b_n & \mbox{si } f(b_n)\cdot f(p_n) < 0 \\
p_n & \mbox{si } f(b_n)\cdot f(p_n) > 0\end{cases}$$

Donde los valores iniciales vienen dados por:

$$a_0 := a, \quad b_0 := b.$$

Se puede probrar que las tres sucesiones convergen al valor de la única raíz del intervalo:

$$ \lim_{n \to \infty} a_n = \lim_{n \to \infty} p_n = \lim_{n \to \infty} b_n.$$

### Ejemplo método de bisección

Para la función $f(x) = x^3-x-2$

In [ ]:
function biseccion(a,b)
    p = (a+b)/2
    f(x) = x^3-x-2
    for i in 1:50
        if  f(a)*f(p) > 0
            a = p
        end
        
        if f(b)*f(p) > 0
            b = p
        end
        p = (a+b)/2
    end
    return p,f(p)
end;

In [ ]:
biseccion(0,2)

# Graficación usando matplotlib

## Graficas usando PyPlot: Ejemplo 1

In [ ]:
using PyPlot

In [ ]:
x=linspace(-10,10,1000);
y=cos.(x);

In [ ]:
plot(x,y) # la sintaxis es muy parecida a matlab

* matplotlib brinda mas opciones:

In [ ]:
xlabel("x")
ylabel("y")
title("Cos(x)")
grid("on")
plot(x,y)

## Graficas usando PyPlot: Ejemplo 2

En este ejemplo se muestra como converge el metodo de los babilonios a la raiz cuadrada deseada.

In [ ]:
list=zeros(30);
S=10000;
x=0.0001;
for i in 1:30
    x=0.5(x+S/x);
    list[i]=x;
end

In [ ]:
x=linspace(0,35,100);
y=S*ones(length(x));

In [ ]:
scatter(1:30,list);
plot(x,y);
xlim(0,30);

## Graficas usando PyPlot: Ejemplo 3

Gráfica que muestra las intersecciones de $x \tan(x)$ y $\sqrt(A-x^2)$

In [ ]:
x=linspace(-10,10,10000);
y1=sqrt.(100.0-x.^2 +im-im);
y2=x.*tan.(x);
y3=y2-y1;

In [ ]:
plot(x,y1)
plot(x,y2)
ylim(0,20)
show()

In [ ]:
plot(x,y3)
ylim(-10,10)
show()

## Graficas usando PyPlot: Ejemplo 4

In [ ]:
"""
Calcula las raices para el pozo finito con A=10.0
"""
function raices_pozo_finito(init::Float64)
    list=zeros(Complex64,100);
    x=init;
    for i in 1:100
        f= x*tan(x)-sqrt(complex(10-x^2))+im -im
        df=complex(x*sec(x)^2+x/sqrt(complex(10.0-x^2))+tan(x)) # Esto lo hice para evitar errores de dominio
        x=x-f/df;
        list[i]=x;
    end
    return list
end;

In [ ]:
list1=raices_pozo_finito(0.1);
list2=raices_pozo_finito(1.1);
list3=raices_pozo_finito(1.5);
list4=raices_pozo_finito(3.5);

In [ ]:
plot(real(list1))
plot(real(list2))
plot(real(list3))
plot(real(list4))
show()

# Usando Julia como CAS llamando a SymPy

Si todavia no contamos con la libreria en nuestra biblioteca, podemos añadirla haciendo:

In [ ]:
using SymPy

Primero definimos las variables simbólicas que vamos a usar:

In [ ]:
A,x,a,n,m=Sym("A,x,a,n,m")

In [ ]:
typeof(n)

Evaluando la suma sugerida en el exámen

In [ ]:
summation(a^n,(n,0,m))

## Cálculo

In [ ]:
integrate(x,x)

In [ ]:
diff(sin(x),x)

Si $x$ es simbólica y llamamos al $\sin$, Julia autómaticamente usa la función apropiada:

In [ ]:
typeof(ans)

In [ ]:
?SymPy.sin

La función que tenian que derivar a mano para el ejercicio del método de Newton-Raphson

In [ ]:
f=simplify(diff(x*tan(x)-sqrt(A-x^2),x))

## Conversión de funciones

El siguiente comando convierte una función de SymPy en una funcion de Julia:

In [ ]:
f2=subs(f,A=>10.0)

Lo que pasa si evaluamos en otra variable simbólica:

In [ ]:
f2(n)

Uno puede evaluar directamente la función en un valor númerico, sin embargo el _output_ sigue siendo un objeto simbólico:

In [ ]:
f2(0.1)

In [ ]:
typeof(ans)

Podemos pasar el resultado a otra función y se sigue tratando como un objeto simbólico:

In [ ]:
cos(f2(0.1))

Con la función _lambify_ podemos convertir una función de SymPy directamente en una función de Julia,típicamente hacer esto nos permite hacer rutinas mas eficientes:

In [ ]:
A=Sym("10.0");fjulia=lambdify(f2,[x]);

In [ ]:
fjulia(0.1)

In [ ]:
typeof(ans)

Podemos comprobar que todo salio bien haciendo la derivada númerica:

In [ ]:
function ff(x)
    A=10.0
    return  x*tan(x)-sqrt(A-x^2)
end

In [ ]:
(ff(0.1+0.001)-ff(0.1))/0.001

Si queremos convertir funciones de Julia en funciones de SymPy, podemos hacer los siguiente:

Primero definimos una función en Julia:

In [ ]:
function r(x)
   return x^2 
end;

Luego, como ya definimos $x$ como una variable simbólica, simplemente evaluamos nuestra función en dicha variable:

In [ ]:
r(x)

In [ ]:
typeof(ans)

Ya podemos, por ejemplo, derivar sin problema:

In [ ]:
diff(r(x),x)

In [ ]:
diff(cos(r(x)),x)

# Evaluación de funciones sobre vectores e intervalos

Ya se comentó en la clase anterior que para pasar arreglos u otros objetos como los que construimos con _range_ y _linspace_ a una función, típicamente bastaba con hacer `f(x)` donde $x$ es un arreglo, sin embargo la versión más reciente de Julia nos pide que lo escribamos como `f.(x)`. Por ej: `cos.(x)`.

Recordemos la función rango:

In [ ]:
typeof(1:3)

Podemos pedir el rango a pasos diferentes:

In [ ]:
a=1:0.02:3
typeof(a)

In [ ]:
b=linspace(1.0,3.0,101)
typeof(b)

In [ ]:
(3-1)/100

In [ ]:
a==b

In [ ]:
last(a)

In [ ]:
last(b)

In [ ]:
length(b)

Evaluamos ahora la función que definimos arriba en un intervalo:

In [ ]:
ff.(linspace(0.0,1.0,100))

# Ejercicio/Tarea 4 Jueves 31 de Agosto

__Utilizar lo que hemos visto hasta ahora de Julia para:__

1. Graficar como convergen $4$ condiciones iniciales a las raiz real de $x^3 - x^2 - x - 1$ utilizando el método de Newton. Ésto básicamente es gráficar como se mueven los puntos $x_k$, luego grafiquen el error relativo a la raiz, es decir $(x_k-S)/S$, donde $S$ es la raiz.

2. Crear una función en Julia que implemente el método de Newton para funciones arbitrarias. Deberán crearla con las siguientes consideraciones:
    * Que tome como argumentos tanto a la función, su derivada y la condición inicial.
    * Hacer la misma función pero que en lugar de tomar como argumento una condición inicial, tome un intervalo, por ejemplo costruido con `linspace`, recordando que hay que llamar a las funciones con un punto `f.(x)`.
    * Combinar las dos formas en una sola rutina, aqui será necesario poner un condicional `if` que verifique de que tipo es el _input_.
    * Finalmente, desarrollen una función similar a la anterior, siguiendo los mismos pasos, pero esta vez que los _inputs_ de la función del método sea solamente la función y la condición (o intervalo) inicial.
        * Para esto deberán usar lo que aprendimos de SymPy y como podemos convertir y/o evaluar funciones tanto en simbolos como en flotantes.
        
__Nota:__ Se resolveran dudas de esta tarea el Martes 29, así que traten de avanzar para que surjan dudas.

# Tarea 5 para el Martes 29 de Agosto

1. Hacer un notebook en su repositorio de Tareas que contenga una breve explicación de como funcioan los siguientes métodos de integración:

* Método de integración de Riemann, es decír, tomando la definición de integral de Riemann con diferencias finitas.
* Regla del trapecio.
* Regla de Simpson de orden 2, es decir, interpolando con polinomios de orden 2.

Pueden consultar Wikipedia.

Usen celdas de _markdown_, recuerden que pueden usar código $\LaTeX$.

# Tarea 5 Viernes 1ro de Septiembre, se dará asesoria el Martes 31 de 8 a 9am

2. Implementar los métodos mencionados en Julia, en el caso de Riemann la rutina tiene que llevar como _input_ los límites y el ancho de los rectangulos que se usarán para aproximar la integral.